<a href="https://colab.research.google.com/github/XavierCachan/moduleIA_S4/blob/main/Regression_lineaire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Retour vers la partie 1 : Quelques notions d'optimisation](https://colab.research.google.com/drive/1ufE9nLrw3ihLsPFJQioR5Sx78IdUmZyq#scrollTo=g0-onQn2OsU-)

# **TP IA Partie 2 - Caractérisation d'une batterie par optimisation - IUT de Cachan GEII2 2024**
XM - Février 2024 - Version : 0.8

-----

Note : Pour avancer dans ce notebook, il suffit d'exécuter (petite flèche), ou compléter puis exécuter, les différents blocs de code placés ci-dessous.

Pour cette deuxième partie, nous cherchons à nous rapprocher de la notion de neurone artificiel, qui est très utilisé pour mettre en place une intelligence artificielle.  

Un étudiant a trouvé une batterie, mais il ne connait pas ses caractéristiques car l'étiquette est illisible. On rappelle qu'un modèle simple d'une batterie est une source de tension E en série avec une résistance interne Ri.

Une batterie peut être modélisée simplement de la manière suivante :

![Modelisation d'un batterie électrique.](https://cabanisbrive.scenari-community.org/STIDD/Premiere/Sequence_07/Exp/Modeliser_une_Batterie_web/res/Model_electrique.png)

Comme vous le savez depuis le S2, le modèle est donc **U_bat = E - Ri * I_bat**, ce qui correspond à l'équation d'une droite y = b + w * x.
Pour déterminer les 2 paramètres E et Ri de la batterie, l'étudiant souhaite automatiser leur recherche à partir d'un ensemble de mesures de tension U_bat réalisées pour différentes charges (donc différents niveaux de courant I_bat).

**1. Import des modules nécessaires pour les calculs et l'affichage**

In [ ]:
import numpy as np                # Module de fonctions mathématiques
import math                       # Module de fonctions mathématiques
import matplotlib.pyplot as plt   # Module de fonctions pour affichage

**2. Fabrication des signaux de mesure de tension U_mes**

Dans cet exercice, on utilisera la notation _th pour la fonction "théorique" et _mod pour la fonction "modèle". Le modèle doit donc retrouver la fonction théorique.

In [ ]:
N = 40                           # Nombre de mesures
I_bat = np.linspace(0,10,num=N)  # Fabrication d'un vecteur de N points régulièrement espacés entre 0 et 10 (0, 0.25, 0.5,...9.75)

# Définition de la fonction mathématique théorique (notée _th). Le modèle que l'on va essayer de retrouver automatiquement est U_bat = E - Ri * I_bat
Ri_th = 0.15                     # Poids théorique à trouver. Par défaut 0.15 ohms
E_th = 12                        # Biais théorique à trouver. Par défaut 12 V
U_th = E_th - Ri_th * I_bat      # Valeurs des tensions si la mesure est "parfaite"

Pour rendre le cas plus réel, on ajoute du bruit de mesure

In [ ]:
sigma = 0.3            # "Intensité" de bruit. Par défaut 0.3
bruit = sigma*(np.random.randn(len(U_th)))
U_mes = U_th + bruit   # Signaux de mesure "réels" = signaux parfaits + bruit

In [ ]:
# Affichage des mesures
fig,ax = plt.subplots(1,figsize=(8,5))
ax.plot (I_bat, U_mes, marker="+", linestyle="none", linewidth=1, label="Tension réelle mesurée")
ax.plot (I_bat, U_th, linestyle="--", linewidth=0.5, label="Droite parfaite (ce que l'on souhaite retrouver)")
ax.set_title("Mesures sur la batterie")
ax.set_xlabel("Entrées Courant I_bat")
ax.set_ylabel("Tension U_bat")
ax.legend()
plt.show()

Dans la partie 1 du TP, l'inconnue était la position X. Ici, il y a deux inconnues qui sont les 2 paramètres b (biais) et w (pente) d'une droite, et on recherche la droite "la plus proche" en moyenne des points de mesure de tension.

**3. Recherche des paramètres de la batterie par optimisation (régression linéaire)**

Comme dans la Partie 1 du TP, il faut définir une fonction coût. On la choisit ici comme la somme des carrés des "distances" (donc les différences) entre les points de mesure $U_i$ et une droite définie par l'équation du modèle y = w * $I_i$ + b (c'est l'équation U_bat = E - Ri * I_bat) .

$cout = \sum\limits_{i=0}^{N-1} {\lVert w * I_i + b - U_i\rVert}^2$

On espère donc trouver en fin d'optimisation la droite "qui colle le mieux", c'est à dire celle qui aura w = -Ri_th et b = E_th.

In [ ]:
# Définition de la fonction coût : on va chercher la droite qui présente la distance moyenne la plus petite par rapport aux points de mesure
def cout(w,b,Iv,Uv,N): # arguments = pente & biais de la droite considérée, vecteur des entrées I_bat & des sorties U_mes, nombre de points de mesures
  somme = 0
  for i in range(0,N):
    somme = somme + (w*Iv[i]+b - Uv[i])**2  # Somme des carrés des distances "verticales"
  return somme/N

On a cette fois 2 variables inconnues : la pente w de la droite et le biais b (l'origine à 0). On définit alors la fonction retournant les dérivées partielles de la fonction coût par rapport à ces 2 variables.

In [ ]:
# Définition de la fonction dérivée (gradient) du coût : si l'on trouve les valeurs qui l'annulent, alors on sera au point de coût minimum
def dcout(w,b,Iv,Uv,N):   # Arguments = pente & biais de la droite considérée, vecteur des entrées I_bat & des sorties U_mes, nombre de points de mesures
  dcW = 0                 # Dérivée partielle par rapport au poids
  dcB = 0                 # Dérivée partielle par rapport au biais
  for i in range(0,N):
    dcW = dcW + 2*Iv[i]*(w*Iv[i]+b - Uv[i])  # Dérivée partielle par rapport à w
    dcB = dcB + 2*(w*Iv[i]+b - Uv[i])        # Dérivée partielle par rapport à b
  return dcW/N, dcB/N

On définit les paramètres de l'optimisation :

In [ ]:
taux_evolution = 0.01   # "Vitesse" d'avancée de l'algorithme. Par défaut 0.01
maxIter = 10000         # Nombre max d'itérations. Par défaut 10000
eps = 0.0001            # Valeur minimale acceptable du gradient. Par défaut 0.0001
iter = 0                # Numéro itération en cours

# Initialisation des paramètres de l'optimisation
w_init = 0              # Pente initiale de la droite
b_init = 0              # Biais initial
U_init = b_init + w_init * I_bat  # Droite de départ pour l'optimisation

grad = 1000             # Valeur initiale du gradient. Par défaut 1000

liste_iter = []         # Listes pour affichage de la convergence de l'erreur
liste_erreur = []
liste_dist = []

Regardons ce qu'il se passe à la première itération :

In [ ]:
gradW, gradB = dcout(w_init,b_init,I_bat,U_mes,N-1) # Recherche de la pente à l'itération 1
grad = math.sqrt(gradW**2 + gradB**2)
w = w_init - taux_evolution * gradW        # On bouge un peu w dans la direction opposée au gradient
b = b_init - taux_evolution * gradB        # On bouge un peu b dans la direction opposée au gradient
iter += 1

# Sauvegarde pour affichage des courbes de pente et distance vs itérations
liste_iter.append(iter)
liste_erreur.append(grad)
liste_dist.append(cout(w,b,I_bat,U_mes,N-1))

# Analyse du modèle obtenu pour notre optimisation à l'itération 1
U_mod = b + w * I_bat

# Affichage des résultats du modèle
fig,ax = plt.subplots(1,figsize=(8,5))
ax.plot (I_bat, U_mes, marker="+", linestyle="none", linewidth=1, label="Tension réelle mesurée")
ax.plot (I_bat, U_th, linestyle="--", linewidth=0.5, label="Droite parfaite (ce que l'on souhaite retrouver)")
ax.plot (I_bat, U_init, linestyle="-", linewidth=1, label="Droite initiale")
ax.plot (I_bat, U_mod, linestyle="-", linewidth=1, label="Droite à l'itération 1")
ax.set_title("Résultats de l'optimisation à l'itération 1")
ax.set_xlabel("Entrées Courant I_bat")
ax.set_ylabel("Tension U_bat")
ax.legend()
plt.show()

print ("Les paramètres théoriques sont : E_th = " + str(E_th) + "V et Ri_th = " + str(Ri_th) + "ohms ")
print ("Les paramètres calculés à l'itération 1 sont : E_mod = " + str(b) + "V et Ri_mod = " + str(-w) + "ohms ")


C'est encore loin d'être parfait... Et si l'on refaisait cela en boucle tant que le gradient (la pente) de la fonction coût n'est pas assez petit ?

In [ ]:
# Boucle d'optimisation
while abs(grad)>eps:
  gradW, gradB = dcout(w,b,I_bat,U_mes,N-1)
  grad = math.sqrt(gradW**2 + gradB**2)
  w = w - taux_evolution * gradW     # On bouge un peu w dans la direction opposée au gradient
  b = b - taux_evolution * gradB     # On bouge un peu b dans la direction opposée au gradient
  iter += 1
  if iter > maxIter:
    grad = 0
  # Sauvegarde pour affichage des courbes de pente et distance vs itérations
  liste_iter.append(iter)
  liste_erreur.append(grad)
  liste_dist.append(cout(w,b,I_bat,U_mes,N-1))

# Affichage de l'évolution du gradient
fig,ax = plt.subplots(1,figsize=(8,5))
ax.plot (liste_iter,liste_erreur, linestyle="-", linewidth=1, label="Gradient")
ax.set_xlabel("Itérations")
ax.set_ylabel("Valeur gradient du coût")
ax.set_xscale('log')
ax.set_yscale('log')
ax.legend()
plt.show()

# Affichage de l'évolution de la distance
fig,ax = plt.subplots(1,figsize=(8,5))
ax.plot (liste_iter,liste_dist, linestyle="-", linewidth=1, label="Distance")
ax.set_xlabel("Itérations")
ax.set_ylabel("Valeur du coût")
ax.set_xscale('log')
ax.set_yscale('log')
ax.legend()
plt.show()

print ("Le coût final obtenu est " + str(cout(w,b,I_bat,U_mes,N-1)))

La courbe de pente du coût semble converger vers 0, c'est bon signe ! Que donne le modèle obtenu ?

In [ ]:
# Analyse du modèle obtenu
U_mod = b + w * I_bat

# Affichage des résultats du modèle
fig,ax = plt.subplots(1,figsize=(8,5))
ax.plot (I_bat, U_mes, marker="+", linestyle="none", linewidth=1, label="Tension réelle mesurée")
ax.plot (I_bat, U_th, linestyle="--", linewidth=0.5, label="Droite parfaite (ce que l'on souhaite retrouver)")
ax.plot (I_bat, U_mod, linestyle="-", linewidth=1, label="Droite optimisée")
ax.set_title("Résultats de l'optimisation")
ax.set_xlabel("Entrées Courant I_bat")
ax.set_ylabel("Tension U_bat")
ax.legend()
plt.show()

print ("Les paramètres théoriques sont : E_th = " + str(E_th) + "V et Ri_th = " + str(Ri_th) + "ohms ")
print ("Les paramètres calculés par le modèle en " + str(iter) + " itérations sont E_mod = " + str(b) + "V et Ri_mod = " + str(-w) + "ohms ")

Que peut-on en conclure ?

Que se passe-t-il si l'on met un taux d'évolution 10 fois plus grand pour accélérer la convergence ?

Ce système linéaire avec un poids et un biais constitue la base d'un neurone artificiel.  
Hum... Est-ce que cela signifie que l'on peut traiter le même problème avec un neurone artificiel ? => [Lien vers la partie 3 : Caractérisation d'une batterie par neurone un artificiel](https://colab.research.google.com/drive/1O0b8AnUaOWrjGoY7RDKRmHjWr2QeJZ_Z#scrollTo=xYl_UBCdxmOC)